In [1]:
import os
from uuid import uuid4
from openai.embeddings_utils import get_embedding
import pinecone
from dotenv import load_dotenv
from tqdm.auto import tqdm
import datetime
from time import sleep
import openai

c:\Users\vtnguye2503\Projects\teli-ai\venv\lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv()
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
INDEX_NAME = "teli-ai"
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDINGS_MODEL = "text-embedding-ada-002"

In [3]:

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
if INDEX_NAME not in pinecone.list_indexes():
    pinecone.create_index(INDEX_NAME, dimension=1536)
index = pinecone.Index(INDEX_NAME)

In [ ]:
testing_results_dir = "asset/testing_result/"
conversations =[] #List of convo and embeddings

for folder_name in os.listdir(testing_results_dir):
    folder_dir = os.path.join(testing_results_dir,folder_name)
    for file in os.listdir(folder_dir):
        if file.endswith(".txt"): #Loop through all files
            with open(os.path.join(folder_dir,file), "r") as f:
                content = f.read()
                conversation = {"conversation":content}
                conversations.append(conversation)

In [8]:
batch_size = 9

for i in tqdm(range(0,len(conversations),batch_size)):
    i_end = min(i+batch_size,len(conversations))
    meta_batch = conversations[i:i_end]
    text = [conversation['conversation'] for conversation in meta_batch]
    ids_batch = [str(uuid4()) for _ in range(len(meta_batch))]
    try:
        res = openai.Embedding.create(input = text, model=EMBEDDINGS_MODEL)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=text, model=EMBEDDINGS_MODEL)
                done = True
            except:
                pass

    embeddings = [conversation['embedding'] for conversation in res['data']]
    to_upsert = list(zip(ids_batch,embeddings,meta_batch))
    index.upsert(vectors=to_upsert)

100%|██████████| 6/6 [00:09<00:00,  1.52s/it]


In [9]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 50}},
 'total_vector_count': 50}